In [ ]:
# check versions and devices
print("python version: ", sys.version)
# print("tf version: ", tf.__version__)
# print("devices: ", tf.config.list_physical_devices())

In [ ]:
# convert annotations to dataframes to visualize classes
from pandas import DataFrame
from scipy.io import loadmat
mat = loadmat('data/info.mat')
cars = DataFrame(mat['class_names'][0])
boxes = DataFrame(mat['annotations'][0])
# print("cars:\n", cars.head())
# print("boxes:\n", boxes.head())

In [ ]:
# organize images into directories
import os
import shutil
imlist = os.listdir("./data/images/")
for i in range(len(imlist)):
    img_num = int(imlist[i].split('.')[0])
    dir_num = int(boxes['class'][img_num-1][0][0]-1)
    img_path = os.path.join("./data/images/", imlist[i])
    dir_path = os.path.join("./data/dataset/", str(dir_num)+"/"+imlist[i])
    shutil.move(img_path, dir_path)

In [ ]:
# create train and validation datasets
import tensorflow as tf

batch_size = 126
img_height = 240
img_width = 360

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "./data/dataset/",
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=True
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "./data/dataset/",
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=True
)

class_names = train_ds.class_names
# TODO: figure out why not sorted
# print("class names: ", class_names)

In [ ]:
# configure performance
autotune = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=autotune)
val_ds = val_ds.cache().prefetch(buffer_size=autotune)

In [ ]:
# model
model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(196)
])

model.summary()



In [ ]:
# compile model
model.compile(
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [ ]:
# train
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=3
)

In [ ]:
# plot accuracy and val_accuracy
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label = 'test')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='upper left')
plt.show()